In [45]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.python.keras import optimizers, losses, activations, models
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.python.keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate, Flatten
from tensorflow.keras.metrics import AUC
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split


In [46]:
df_1 = pd.read_csv("C:/00ETH/ml4h/Project1/archive/ptbdb_normal.csv", header=None)
df_2 = pd.read_csv("C:/00ETH/ml4h/Project1/archive/ptbdb_abnormal.csv", header=None)
df = pd.concat([df_1, df_2])

df_train, df_test = train_test_split(df, test_size=0.2, random_state=1337, stratify=df[187])


Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]

In [47]:
AUCROC = AUC(curve='ROC')
AUCPR = AUC(curve='PR')

In [48]:
def identity_block(x,filter):
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv1D(filter[0], kernel_size=5, padding = 'same')(x)
    x = tf.keras.layers.Conv1D(filter[0], kernel_size=5, padding = 'same')(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv1D(filter[1], kernel_size=3, padding = 'same')(x)
    x = tf.keras.layers.Conv1D(filter[1], kernel_size=3, padding = 'same')(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [49]:
def convolutional_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv1D(filter[0], kernel_size=3, padding = 'same')(x)
    x = tf.keras.layers.Conv1D(filter[0], kernel_size=3, padding = 'same')(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv1D(filter[1], kernel_size=3, padding = 'same')(x)
    x = tf.keras.layers.Conv1D(filter[1], kernel_size=3, padding = 'same')(x)
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv1D(filter[1], kernel_size=3, padding = 'same')(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [54]:
def get_model1():
    nclass = 1
    inp = Input(shape=(187, 1))
    img_1 = tf.keras.layers.Convolution1D(16, kernel_size=5, activation=activations.relu, padding="same")(inp)
    img_1 = tf.keras.layers.Convolution1D(16, kernel_size=5, activation=activations.relu, padding="same")(img_1)
    img_1 = tf.keras.layers.MaxPool1D(pool_size=2)(img_1)
    img_1 = tf.keras.layers.Dropout(rate=0.1)(img_1)
    
    img_shortcut = convolutional_block(img_1,[32,32])
    img_shortcut = identity_block(img_shortcut,[32,32])
    
    img_1 = tf.keras.layers.Conv1D(256, kernel_size=3, padding = 'same')(img_shortcut)
    img_1 = tf.keras.layers.Conv1D(256, kernel_size=3, padding = 'same')(img_1)
    img_1 = tf.keras.layers.Activation('relu')(img_1)
    img_1 = Flatten()(img_1)
    
    dense_1 = tf.keras.layers.Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = tf.keras.layers.Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = tf.keras.layers.Dense(nclass, activation=activations.softmax, name="dense_3_mitbih")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=[AUCROC, AUCPR, 'acc'])
    model.summary()
    return model

In [55]:
model1 = get_model1()
file_path = "Resnet_cnn_ptbdb1.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 187, 1)]     0                                            
__________________________________________________________________________________________________
conv1d_444 (Conv1D)             (None, 187, 16)      96          input_12[0][0]                   
__________________________________________________________________________________________________
conv1d_445 (Conv1D)             (None, 187, 16)      1296        conv1d_444[0][0]                 
__________________________________________________________________________________________________
max_pooling1d_11 (MaxPooling1D) (None, 93, 16)       0           conv1d_445[0][0]                 
____________________________________________________________________________________________

In [56]:
model1.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model1.load_weights(file_path)

pred_test = model1.predict(X_test)
pred_test = (pred_test>0.5).astype(np.int8)

Train on 10476 samples, validate on 1165 samples
Epoch 1/1000

Epoch 00001: val_acc improved from -inf to 0.69957, saving model to Resnet_cnn_ptbdb1.h5
10476/10476 - 26s - loss: 4.2256 - auc_4: 0.5000 - auc_5: 0.7244 - acc: 0.7244 - val_loss: 4.6066 - val_auc_4: 0.5000 - val_auc_5: 0.6996 - val_acc: 0.6996
Epoch 2/1000

Epoch 00002: val_acc did not improve from 0.69957
10476/10476 - 19s - loss: 4.2256 - auc_4: 0.5000 - auc_5: 0.7244 - acc: 0.7244 - val_loss: 4.6066 - val_auc_4: 0.5000 - val_auc_5: 0.6996 - val_acc: 0.6996
Epoch 3/1000

Epoch 00003: val_acc did not improve from 0.69957
10476/10476 - 20s - loss: 4.2256 - auc_4: 0.5000 - auc_5: 0.7244 - acc: 0.7244 - val_loss: 4.6066 - val_auc_4: 0.5000 - val_auc_5: 0.6996 - val_acc: 0.6996
Epoch 4/1000

Epoch 00004: val_acc did not improve from 0.69957

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
10476/10476 - 22s - loss: 4.2256 - auc_4: 0.5000 - auc_5: 0.7244 - acc: 0.7244 - val_loss: 4.6066 - val_au

In [30]:
f1 = f1_score(Y_test, pred_test, average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.4193097945342111 
Test accuracy score : 0.7220886293369976 


In [57]:
def get_model2():
    nclass = 1
    inp = Input(shape=(187, 1))
    img_1 = tf.keras.layers.Convolution1D(16, kernel_size=5, activation=activations.relu, padding="same")(inp)
    img_1 = tf.keras.layers.Convolution1D(16, kernel_size=5, activation=activations.relu, padding="same")(img_1)
    img_1 = tf.keras.layers.MaxPool1D(pool_size=2)(img_1)
    img_1 = tf.keras.layers.Dropout(rate=0.1)(img_1)
    
    img_shortcut = convolutional_block(img_1,[32,32])
        
    img_1 = tf.keras.layers.Conv1D(256, kernel_size=3, padding = 'same')(img_shortcut)
    img_1 = tf.keras.layers.Conv1D(256, kernel_size=3, padding = 'same')(img_1)
    img_1 = tf.keras.layers.Activation('relu')(img_1)
    img_1 = GlobalMaxPool1D()(img_1)

    dense_1 = tf.keras.layers.Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = tf.keras.layers.Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = tf.keras.layers.Dense(nclass, activation=activations.softmax, name="dense_3_mitbih")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=[AUCROC,AUCPR, 'acc'])
    model.summary()
    return model

In [58]:
model2 = get_model2()
file_path = "Resnet_cnn_ptbdb2.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 187, 1)]     0                                            
__________________________________________________________________________________________________
conv1d_457 (Conv1D)             (None, 187, 16)      96          input_13[0][0]                   
__________________________________________________________________________________________________
conv1d_458 (Conv1D)             (None, 187, 16)      1296        conv1d_457[0][0]                 
__________________________________________________________________________________________________
max_pooling1d_12 (MaxPooling1D) (None, 93, 16)       0           conv1d_458[0][0]                 
____________________________________________________________________________________________

In [59]:
model2.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model2.load_weights(file_path)

pred_test2 = model2.predict(X_test)
pred_test2 = (pred_test2>0.5).astype(np.int8)

Train on 10476 samples, validate on 1165 samples
Epoch 1/1000

Epoch 00001: val_acc improved from -inf to 0.69957, saving model to Resnet_cnn_ptbdb2.h5
10476/10476 - 21s - loss: 4.2256 - auc_4: 0.5000 - auc_5: 0.7244 - acc: 0.7244 - val_loss: 4.6066 - val_auc_4: 0.5000 - val_auc_5: 0.6996 - val_acc: 0.6996
Epoch 2/1000

Epoch 00002: val_acc did not improve from 0.69957
10476/10476 - 13s - loss: 4.2256 - auc_4: 0.5000 - auc_5: 0.7244 - acc: 0.7244 - val_loss: 4.6066 - val_auc_4: 0.5000 - val_auc_5: 0.6996 - val_acc: 0.6996
Epoch 3/1000

Epoch 00003: val_acc did not improve from 0.69957
10476/10476 - 15s - loss: 4.2256 - auc_4: 0.5000 - auc_5: 0.7244 - acc: 0.7244 - val_loss: 4.6066 - val_auc_4: 0.5000 - val_auc_5: 0.6996 - val_acc: 0.6996
Epoch 4/1000

Epoch 00004: val_acc did not improve from 0.69957

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
10476/10476 - 17s - loss: 4.2256 - auc_4: 0.5000 - auc_5: 0.7244 - acc: 0.7244 - val_loss: 4.6066 - val_au

In [34]:
f1 = f1_score(Y_test, pred_test2, average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test2)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.4193097945342111 
Test accuracy score : 0.7220886293369976 


In [62]:
def get_model3():
    nclass = 1
    inp = Input(shape=(187, 1))
    img_1 = tf.keras.layers.Convolution1D(16, kernel_size=5, activation=activations.relu, padding="same")(inp)
    img_1 = tf.keras.layers.Convolution1D(16, kernel_size=5, activation=activations.relu, padding="same")(img_1)
    img_1 = tf.keras.layers.MaxPool1D(pool_size=2)(img_1)
    img_1 = tf.keras.layers.Dropout(rate=0.1)(img_1)
    
    img_1 = tf.keras.layers.Convolution1D(32, kernel_size=5, activation=activations.relu, padding="same")(img_1)
    img_1 = tf.keras.layers.Convolution1D(32, kernel_size=5, activation=activations.relu, padding="same")(img_1)
    img_1 = tf.keras.layers.MaxPool1D(pool_size=2)(img_1)
    img_1 = tf.keras.layers.Dropout(rate=0.1)(img_1)
    
    img_shortcut = identity_block(img_1,[32,32])
    
    img_1 = tf.keras.layers.Conv1D(256, kernel_size=3, padding = 'same')(img_shortcut)
    img_1 = tf.keras.layers.Conv1D(256, kernel_size=3, padding = 'same')(img_1)
    img_1 = tf.keras.layers.Activation('relu')(img_1)
    img_1 = GlobalMaxPool1D()(img_1)

    dense_1 = tf.keras.layers.Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = tf.keras.layers.Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = tf.keras.layers.Dense(nclass, activation=activations.softmax, name="dense_3_mitbih")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=[AUCROC,AUCPR, 'acc'])
    model.summary()
    return model

In [63]:
model3 = get_model3()
file_path = "Resnet_cnn_ptbdb3.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           [(None, 187, 1)]     0                                            
__________________________________________________________________________________________________
conv1d_470 (Conv1D)             (None, 187, 16)      96          input_15[0][0]                   
__________________________________________________________________________________________________
conv1d_471 (Conv1D)             (None, 187, 16)      1296        conv1d_470[0][0]                 
__________________________________________________________________________________________________
max_pooling1d_15 (MaxPooling1D) (None, 93, 16)       0           conv1d_471[0][0]                 
____________________________________________________________________________________________

In [64]:
model3.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model3.load_weights(file_path)

pred_test3 = model3.predict(X_test)
pred_test3 = (pred_test3>0.5).astype(np.int8)

Train on 10476 samples, validate on 1165 samples
Epoch 1/1000

Epoch 00001: val_acc improved from -inf to 0.69957, saving model to Resnet_cnn_ptbdb3.h5
10476/10476 - 18s - loss: 4.2256 - auc_4: 0.5000 - auc_5: 0.7244 - acc: 0.7244 - val_loss: 4.6066 - val_auc_4: 0.5000 - val_auc_5: 0.6996 - val_acc: 0.6996
Epoch 2/1000

Epoch 00002: val_acc did not improve from 0.69957
10476/10476 - 10s - loss: 4.2256 - auc_4: 0.5000 - auc_5: 0.7244 - acc: 0.7244 - val_loss: 4.6066 - val_auc_4: 0.5000 - val_auc_5: 0.6996 - val_acc: 0.6996
Epoch 3/1000

Epoch 00003: val_acc did not improve from 0.69957
10476/10476 - 10s - loss: 4.2256 - auc_4: 0.5000 - auc_5: 0.7244 - acc: 0.7244 - val_loss: 4.6066 - val_auc_4: 0.5000 - val_auc_5: 0.6996 - val_acc: 0.6996
Epoch 4/1000

Epoch 00004: val_acc did not improve from 0.69957

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
10476/10476 - 12s - loss: 4.2256 - auc_4: 0.5000 - auc_5: 0.7244 - acc: 0.7244 - val_loss: 4.6066 - val_au

In [65]:
f1 = f1_score(Y_test, pred_test3, average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test3)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.4193097945342111 
Test accuracy score : 0.7220886293369976 
